In [ ]:
import os
import sqlite3
import xml.etree.ElementTree as ET
import pandas as pd
import requests

start = 0  # начальный шаг
step = 50  # шаг парсинга

while True:
    url = f"https://iss.moex.com/iss/sitenews.xml?start={start}&from=2020-01-01&to=2025-01-01"
    response = requests.get(url)
    root = ET.fromstring(response.content)
    rows = root.findall("./data/rows/row")
    if not rows:
        break

    start += step
    print(start)
    dfs = {}  # словарь для хранения dataframe

    for data in root.findall("./data"):  # поиск всех элементов data
        id = data.attrib["id"]  # получение атрибута id
        columns = [
            c.attrib["name"] for c in data.findall("./metadata/columns/column")
        ]  # список колонок
        df = pd.DataFrame(columns=columns)  # создание пустого dataframe с определенными колонками
        rows = data.findall("./rows/row")  # поиск строк

        for row in rows:
            data = dict(
                zip(columns, [row.attrib.get(c) for c in columns])
            )  # словарь данных строки
            df = pd.concat(
                [df, pd.DataFrame([data])], ignore_index=True
            )  # конкатенация строк

        dfs[id] = df  # сохранение dataframe в словаре

    # далее код сохранения данных в БД и CSV

    if not os.path.exists("moexmetadata"):
        os.mkdir("moexmetadata")

    conn = sqlite3.connect("moexmetadata.db")

    for id, df in dfs.items():
        df.to_sql(id, conn, if_exists="append", index=False)

        # Проверка наличия колонки id

        if "id" in df.columns:
            cursor = conn.cursor()

            # Проверяем наличие индекса

            cursor.execute(f"PRAGMA index_info('index_name')")

            index_info = cursor.fetchall()

            if not index_info:
                # если индекса нет, то создаем индекс в БД в таблице по колонке id

                cursor.execute(f"CREATE INDEX index_name ON {id} (id)")
                conn.commit()

        # данные дублируем в файл
        filename = f"{id}.txt"
        path = os.path.join("moexmetadata", filename)
        with open(path, "a", encoding="utf-8") as f:
            df.to_csv(f, sep="\t", index=False)

conn.close()


Выходные данные были обрезаны до нескольких последних строк (5000).
422850
422900
422950
423000
423050
423100
423150
423200
423250
423300
423350
423400
423450
423500
423550
423600
423650
423700
423750
423800
423850
423900
423950
424000
424050
424100
424150
424200
424250
424300
424350
424400
424450
424500
424550
424600
424650
424700
424750
424800
424850
424900
424950
425000
425050
425100
425150
425200
425250
425300
425350
425400
425450
425500
425550
425600
425650
425700
425750
425800
425850
425900
425950
426000
426050
426100
426150
426200
426250
426300
426350
426400
426450
426500
426550
426600
426650
426700
426750
426800
426850
426900
426950
427000
427050
427100
427150
427200
427250
427300
427350
427400
427450
427500
427550
427600
427650
427700
427750
427800
427850
427900
427950
428000
428050
428100
428150
428200
428250
428300
428350
428400
428450
428500
428550
428600
428650
428700
428750
428800
428850
428900
428950
429000
429050
429100
429150
429200
429250
429300
429350
429400
429450
4

In [ ]:
import pandas as pd

df = pd.read_csv('/content/moexmetadata/sitenews.txt')
df.head()

,id,tag,title,published_at,modified_at
0,67615,site,"ООО &quot;НТ Прогресс&quot;, дочерняя компания...",2024-02-15 19:00:00,2024-02-15 19:00:04
1,67614,site,16.02.2024 состоится депозитный аукцион Федера...,2024-02-15 18:25:21,2024-02-15 18:25:22
2,67613,site,О начале торгов ценными бумагами &quot;16&quot...,2024-02-15 18:23:44,2024-02-15 20:08:46
3,67612,site,Определена цена исполнения фьючерсного контракта,2024-02-15 18:22:44,2024-02-15 18:22:44
4,67611,site,"О регистрации выпуска биржевых облигаций, о вк...",2024-02-15 18:22:07,2024-02-15 18:22:07
